# Evals from Challenge Module 9

## Description

This Jupyter notebook demonstrates multiple runs of the planner agent from `run.py` to evaluate its behavior according to the challenge requirements. Creating deterministic tests for a planner agent is challenging, especially when we need to test specific behaviors like:

- **Planner (B)**: Steps advance only when `success_criteria` is true
- **Retries**: Each step can be retried up to `MAX_RETRIES` times
- **HITL (Human-in-the-Loop)**: Switch that controls when the agent requires human approval to resume

By running multiple scenarios and observing the outputs, we can verify that the planner agent correctly implements these core requirements of the challenge.

Each run below demonstrates different aspects of the planner's behavior and includes explanations of what happened in relation to the challenge requirements.

In [2]:
from main import main

In [ ]:
# Run the planner agent script to capture its output
main(['y'])

User Input: What is 2 + 2?

=== Execution ===
planner: step=0, retries=0
  Plan created:
    0: validate [HITL]
    1: lookup
    2: calculate
    3: analyze
__interrupt__: executed

🛑 INTERRUPTED - Pending: ('executor',)
Response: y
executor: step=0, retries=0 -> validate (Validate information accuracy and consistency)
advance_cursor: step=1, retries=0
executor: step=1, retries=0 -> lookup (Lookup additional context from reference materials)
advance_cursor: step=2, retries=0
executor: step=2, retries=0 -> calculate (Perform calculations on retrieved data)
advance_cursor: step=3, retries=0
executor: step=2, retries=0 -> calculate (Perform calculations on retrieved data)
finalize: step=0, retries=0

✅ Execution complete


## How HITL Worked in This Execution

Based on the execution output above, here's how HITL (Human-in-the-Loop) functioned:

### HITL in Action:

**1. Plan with HITL Step:**
```
Plan created:
  0: validate [HITL]  <- This step requires human approval
  1: lookup
  2: calculate
  3: analyze
```

**2. Execution Interruption:**
- The planner created step 0 as a validation step marked `[HITL]`
- When the executor reached step 0, it detected `requires_interrupt=True`
- Execution was **interrupted**: `🛑 INTERRUPTED - Pending: ('executor',)`
- The system waited for human input

**3. Human Approval & Resume:**
- User provided input: `Response: y` (approving the step)
- **Key**: The system used the same `thread_id` to resume execution from the exact interruption point
- This set the `human_approved` flag in the state scratch pad
- Execution resumed immediately after approval using the preserved state

**4. Continued Execution:**
- After approval, step 0 executed: `executor: step=0, retries=0 -> validate`
- The workflow continued normally through the remaining steps
- No further interruptions occurred since steps 1-3 didn't require HITL

**Key HITL Behavior Demonstrated:**
- ✅ Execution paused automatically when reaching HITL step
- ✅ Human approval was required to continue
- ✅ **Thread ID preserved state across interruption/resume cycle**
- ✅ Once approved, execution resumed from exact interruption point
- ✅ Non-HITL steps executed without interruption